# Tesing Bulk dataset - GSE53481

Here we tested the dataset available at accession number: [GSE53481](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE53481). Link to Paper: [Cell-Cycle Control of Developmentally Regulated Transcription Factors Accounts for Heterogeneity in Human Pluripotent Cells](https://doi.org/10.1016/j.stemcr.2013.10.009)

<div id="toc"></div>

## Neccessary Imports

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [8]:
import sys
code = "./../../code/"
data = "./../../data/"
sys.path.append(code)
import pandas
import pypairs as pairs
from sklearn.preprocessing import QuantileTransformer
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import numpy as np
from pathlib import Path
from tqdm import tqdm_notebook as tqdm
import helper
import timeit

init_notebook_mode(connected=True)

## Loading Oscope CC only marker pairs

In [3]:
cc_marker = helper.load_ocope_marker(data)

[__set_matrix] Original Matrix 'x' has shape 19084 x 247
[__set_matrix] Removed 16689 genes that were not in 'subset_genes'. 2395 genes remaining.
[__set_matrix] Removed 61 genes that were not expressed in any samples. 2334 genes remaining.
[__set_matrix] Removed 0 samples that were not annotated in 'phases'. 247 samples remaining.
[__set_matrix] Matrix truncation done. Working with 2334 genes for 247 samples.
[sandbag] Identifying marker pairs...Processing in parallel with 10 processes...
 Done!
[sandbag] Identified 8146 marker pairs (phase: count): {'G1': 2575, 'S': 4101, 'G2M': 1470}


## Loading human embryonic stem cells GSE53481

In [5]:
gencounts_GSE53481 = pandas.read_csv(Path(data + "GSE53481_humanRNAseq.txt"), sep='\t')
genes = [s[s.rindex('_') +1:] for s in gencounts_GSE53481["GENE"]]
gencounts_GSE53481["GENE"] = genes
gencounts_GSE53481.set_index("GENE", inplace=True)
gencounts_GSE53481.head(10)

,H1.DN,H1.KO2,H1.AzLow,H1.AzHigh,H2.DN,H2.KO2,H2.AzLow,H2.AzHigh,H3.DN,H3.KO2,H3.AzLow,H3.AzHigh
GENE,,,,,,,,,,,,
LOC100289255,0.00,0.00,0.02,0.02,0.00,0.01,0.03,0.03,0.00,0.01,0.02,0.03
LOC644656,1.76,1.81,0.73,0.54,2.26,1.71,0.54,0.78,1.63,1.80,1.08,0.64
LOC646903,1.49,1.14,0.61,0.35,1.94,2.01,0.72,0.65,1.12,1.42,0.69,0.44
FLJ36644,0.11,0.12,0.00,0.01,0.12,0.20,0.07,0.03,0.15,0.16,0.12,0.05
LOC284454,0.50,0.99,0.19,0.22,0.85,0.72,0.38,0.40,0.66,0.81,0.36,0.58
LOC149773,0.01,0.00,0.03,0.01,0.03,0.01,0.06,0.02,0.07,0.02,0.04,0.03
LOC100131176,0.21,0.26,0.00,0.00,0.34,0.56,0.20,0.00,1.00,0.38,0.35,0.00
LOC100131366,0.19,0.03,0.31,0.29,0.00,0.30,0.96,0.71,0.30,0.00,0.71,0.94
FLJ42351,0.10,0.06,0.22,0.33,0.15,0.00,0.25,0.18,0.05,0.15,0.25,0.70


In [9]:
x = gencounts_GSE53481.T.values

X_std = QuantileTransformer().fit_transform(x.astype(float))

gencounts_GSE53481_Qnorm = pandas.DataFrame(X_std.T, index=gencounts_GSE53481.index, columns=gencounts_GSE53481.columns)

gencounts_GSE53481_Qnorm.head(10)

,H1.DN,H1.KO2,H1.AzLow,H1.AzHigh,H2.DN,H2.KO2,H2.AzLow,H2.AzHigh,H3.DN,H3.KO2,H3.AzLow,H3.AzHigh
GENE,,,,,,,,,,,,
LOC100289255,1.000000e-07,1.000000e-07,6.361361e-01,6.361361e-01,1.000000e-07,4.194194e-01,9.999999e-01,9.999999e-01,1.000000e-07,4.194194e-01,0.636136,9.999999e-01
LOC644656,7.273273e-01,9.089138e-01,2.728729e-01,1.000000e-07,9.999999e-01,6.364466e-01,1.000000e-07,3.634222e-01,5.455312e-01,8.185142e-01,0.454506,1.818422e-01
LOC646903,8.178995e-01,6.357958e-01,1.820635e-01,1.000000e-07,9.094977e-01,9.999999e-01,4.544619e-01,2.727273e-01,5.455406e-01,7.275551e-01,0.363697,9.079514e-02
FLJ36644,4.547405e-01,6.361361e-01,1.000000e-07,9.078309e-02,6.361361e-01,9.999999e-01,3.635214e-01,1.818182e-01,8.184500e-01,9.089616e-01,0.636136,2.727273e-01
LOC284454,4.545657e-01,9.999999e-01,1.000000e-07,9.058149e-02,9.089687e-01,7.271716e-01,2.727273e-01,3.634332e-01,6.364169e-01,8.183809e-01,0.182107,5.454545e-01
LOC149773,1.816817e-01,1.000000e-07,6.361361e-01,1.816817e-01,6.361361e-01,1.816817e-01,9.092169e-01,4.094094e-01,9.999999e-01,4.094094e-01,0.818040,6.361361e-01
LOC100131176,4.544741e-01,5.454075e-01,1.000000e-07,1.000000e-07,6.365918e-01,9.089923e-01,3.642466e-01,1.000000e-07,9.999999e-01,8.179049e-01,0.727013,1.000000e-07
LOC100131366,2.728443e-01,1.814285e-01,6.356982e-01,3.641536e-01,1.000000e-07,5.000000e-01,9.999999e-01,7.727728e-01,5.000000e-01,1.000000e-07,0.772773,9.096284e-01
FLJ42351,2.726727e-01,1.814858e-01,6.364169e-01,9.089548e-01,4.094094e-01,1.000000e-07,7.727728e-01,5.454278e-01,9.104850e-02,4.094094e-01,0.772773,9.999999e-01


In [11]:
GSE53481_prediction = pairs.cyclone(gencounts_GSE53481_Qnorm, cc_marker, min_pairs=1, verbose=True)

[__set_matrix] Original Matrix 'x' has shape 510 x 12
[__set_matrix] Matrix truncation done. Working with 510 genes for 12 samples.
[cyclone] Preparing marker pairs, where at least one gene was not present in 'x'... Done!
[cyclone] Removed 8102 marker pairs. 8146 marker pairs remaining.
[cyclone] Calculating scores and predicting cell cycle phase... Done!
[cyclone] Calculated scores and prediction (phase: count): G1: 5, S: 2, G2M: 5


In [12]:
GSE53481_prediction_table = helper.get_prediction_table(GSE53481_prediction)
helper.DataTable(GSE53481_prediction_table)

,G1,G2M,S,G1_norm,G2M_norm,S_norm,prediction
sample,,,,,,,
H1.DN,0.606212,0.215075,0.657603,0.409910,0.145430,0.444660,G1
H1.KO2,0.659639,0.099000,0.751773,0.436728,0.065545,0.497727,G1
H1.AzLow,0.429719,0.174174,0.798193,0.306485,0.124225,0.569290,S
H1.AzHigh,0.082495,0.925852,0.027190,0.079664,0.894079,0.026257,G2M
H2.DN,0.956827,0.174349,0.337011,0.651707,0.118751,0.229542,G1
H2.KO2,0.854985,0.001014,0.644535,0.569787,0.000676,0.429537,G1
H2.AzLow,0.496994,0.751503,0.324899,0.315873,0.477631,0.206495,G2M
H2.AzHigh,0.070288,0.988978,0.000000,0.066355,0.933645,0.000000,G2M
H3.DN,0.674372,0.586760,0.096970,0.496555,0.432044,0.071401,G1


## Plot prediction quality

In [14]:
GSE53481_labels = ['G1', 'G1','S','G2M','G1', 'G1','S','G2M','G1', 'G1','S','G2M']

In [15]:
GSE53481_evaluation = helper.evaluate_prediction(GSE53481_prediction_table, GSE53481_labels)

F1 Score: G1: 0.9090909090909091, S: 0.4, G2M: 0.7499999999999999
Reacall: G1: 0.8333333333333334, S: 0.3333333333333333, G2M: 1.0 
Precision: G1: 1.0, S: 0.5, G2M: 0.6 


In [16]:
iplot(helper.plot_evaluation(*GSE53481_evaluation, xaxis=["G1","S","G2M"], xaxislbl="Phase"))

In [18]:
sample1_g1 = [GSE53481_prediction_table.iloc[i, 0] for i in range (0,4)]
sample1_s = [GSE53481_prediction_table.iloc[i, 2] for i in range (0,4)]
sample1_g2m = [GSE53481_prediction_table.iloc[i, 1] for i in range (0,4)]
plot = helper.get_prediction_plot(sample1_g1, sample1_s, sample1_g2m, t="pie", xaxis=['DN', 'KO2', 'AzLow', 'AzHigh'], xaxislbl="H1", title="Assignment of hESC H1 cells",width=950,height=950)
iplot(plot)

In [19]:
avg_g1 = [np.average(GSE53481_prediction_table.iloc[[i, i+4, i+8], 0].values) for i in range (0,4)]
avg_s = [np.average(GSE53481_prediction_table.iloc[[i, i+4, i+8], 2].values) for i in range (0,4)]
avg_g2m = [np.average(GSE53481_prediction_table.iloc[[i, i+4, i+8], 1].values) for i in range (0,4)]
plot = helper.get_prediction_plot(avg_g1, avg_s, avg_g2m, t="pie", xaxis=['DN', 'KO2', 'AzLow', 'AzHigh'], xaxislbl="Average", title="Average assignment of hESC all cells", width=950,height=950)
iplot(plot)